In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
tqdm.pandas()
import os
import gc
import random
from glob import glob
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import warnings
import seaborn as sns
import pickle
import json
import re
import time
import sys
from requests import get
import multiprocessing
import joblib
!pip uninstall lightgbm -y
!pip install lightgbm==3.3.1
!pip uninstall transformers -y
!pip install transformers==4.18.0

import torch
import transformers
from transformers import BertTokenizer

class CFG:
    seed = 46
    target = "point_of_interest"
    n_neighbors = 10
    n_splits = 3

    expID = ""
    if "google.colab" in sys.modules:
        expID = get("http://172.28.0.2:9000/api/sessions").json()[0]["name"].split(".")[0].split("-")[0]

random.seed(CFG.seed)
os.environ["PYTHONHASHSEED"] = str(CFG.seed)
np.random.seed(CFG.seed)

plt.rcParams["font.size"] = 13
warnings.filterwarnings('ignore')

%cd /content/drive/MyDrive/Kaggle/Foursquare/Notebook

Found existing installation: lightgbm 3.3.1
Uninstalling lightgbm-3.3.1:
  Successfully uninstalled lightgbm-3.3.1
  Using cached lightgbm-3.3.1-py3-none-manylinux1_x86_64.whl (2.0 MB)
Found existing installation: transformers 4.18.0
Uninstalling transformers-4.18.0:
  Successfully uninstalled transformers-4.18.0
  Using cached transformers-4.18.0-py3-none-any.whl (4.0 MB)
/content/drive/MyDrive/Kaggle/Foursquare/Notebook


In [ ]:
train = pd.read_csv("../Input/train.csv")
test = pd.read_csv("../Input/test.csv")
test[CFG.target] = "TEST"

train.head(1)

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
0,E_000001272c6c5d,Café Stad Oudenaarde,50.859975,3.634196,Abdijstraat,Nederename,Oost-Vlaanderen,9700,BE,NaN,NaN,Bars,P_677e840bb6fc7e


In [ ]:
train_name_feature = pd.read_csv('../Dataset/train_name_feature_bert.csv')
test_name_feature = pd.read_csv('../Dataset/test_name_feature_bert.csv')

In [ ]:
train_name_feature.shape

(1138812, 769)

In [6]:
train_name_feature_corr = train_name_feature.corr()

In [13]:
train_name_feature_corr

,Unnamed: 0,name_0,name_1,name_2,name_3,name_4,name_5,name_6,name_7,name_8,...,name_758,name_759,name_760,name_761,name_762,name_763,name_764,name_765,name_766,name_767
Unnamed: 0,1.000000,-0.001421,0.000113,0.000224,0.000328,-0.001696,-0.000392,0.000394,-0.000408,0.000649,...,-0.001954,-0.001201,-0.000039,0.000229,0.000184,0.001314,-0.000338,-0.001191,-0.001039,0.000652
name_0,-0.001421,1.000000,0.128597,-0.127400,0.246373,0.071473,0.494635,-0.073163,0.138031,0.138601,...,-0.090373,-0.002688,0.133789,0.438958,0.383662,-0.188011,-0.476281,0.099537,-0.064031,0.056465
name_1,0.000113,0.128597,1.000000,-0.142003,-0.124449,-0.132704,0.019929,0.243773,0.176943,-0.016726,...,0.074354,-0.091496,-0.137409,0.197087,-0.051510,-0.110344,-0.082543,0.187510,0.086859,0.186221
name_2,0.000224,-0.127400,-0.142003,1.000000,0.010176,0.181962,-0.070930,-0.058436,-0.085522,0.066050,...,-0.047981,0.180967,0.028677,-0.240073,-0.057179,0.119160,0.031472,-0.112248,0.049561,-0.269206
name_3,0.000328,0.246373,-0.124449,0.010176,1.000000,0.090817,0.243675,-0.218845,0.073230,0.054269,...,-0.036365,0.182331,0.157490,0.240852,0.099236,0.130712,-0.264719,0.026553,0.046193,0.009361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
name_763,0.001314,-0.188011,-0.110344,0.119160,0.130712,-0.213868,-0.172862,-0.147184,-0.032971,0.062502,...,0.027309,0.267778,-0.038713,-0.164722,-0.223380,1.000000,0.012549,-0.062260,-0.072084,0.082188
name_764,-0.000338,-0.476281,-0.082543,0.031472,-0.264719,-0.102613,-0.417246,0.133531,-0.250952,-0.152506,...,0.040707,-0.219212,-0.192927,-0.290900,-0.259522,0.012549,1.000000,-0.201340,0.060598,-0.057209
name_765,-0.001191,0.099537,0.187510,-0.112248,0.026553,-0.045345,0.133876,0.220200,0.155663,0.052414,...,-0.007467,0.038702,-0.117136,0.039974,0.033560,-0.062260,-0.201340,1.000000,0.033346,0.247183
name_766,-0.001039,-0.064031,0.086859,0.049561,0.046193,0.079232,0.045124,0.074451,0.000731,-0.179489,...,0.011680,-0.089789,0.083112,0.166887,-0.051481,-0.072084,0.060598,0.033346,1.000000,-0.079295
